# Final Project — Artificial Intelligence
## Basketball Player Detection with RFDETR

This project implements an end-to-end workflow for training, evaluating, and deploying an RFDETR object detection model.  
The goal is to detect basketball players in real game scenarios using a dataset sourced from Roboflow and a custom inference application built with Gradio.

**Team:** Andres Restrepo Botero, Juan Jose Sanchez Sanchez, Samuel Lopera Torres  
**Edited:** 2025-03-10  
**Python:** 3.11+  
**Dependencies:** PyTorch ≥ 2.0, torchvision ≥ 0.15, rfdetr, OpenCV, Gradio


### 1. Download Dataset from Roboflow

This section connects to Roboflow, selects the basketball-players project, and downloads the dataset in COCO format for training.


In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="Lyp2S7pSnGzPnFjAkZ9v")
project = rf.workspace("aieafit").project("basketball-players-fy4c2-uzva7")
dataset = project.version(1).download("coco")

loading Roboflow workspace...
loading Roboflow project...
Exporting format coco in progress : 85.0%
Version export complete for coco format



Extracting Dataset Version Zip to Basketball-Players-1 in coco:: 100%|██████████| 406/406 [00:00<00:00, 1652.55it/s]


Note: In production environments, the API key should not be written directly in the notebook. Store it as an environment variable and load it using `os.environ`.


### 2. Model Definition and Training

Here we create an RFDETR model instance and train it using the downloaded dataset. The training configuration (epochs, batch size, learning rate, etc.) is specified in the `model.train` call.


Training parameters:
- `dataset_dir='Basketball-Players-1'`: Folder containing the COCO dataset from Roboflow.
- `epochs=10`: Number of full training passes through the dataset.
- `batch_size=4` with `grad_accum_steps=4`: Simulates a larger effective batch size.
- `lr=1e-4`: Learning rate.
- `output_dir='Basket-Model'`: Directory where checkpoints and logs are saved.


In [1]:
from rfdetr import RFDETRBase

model = RFDETRBase()

model.train(
    dataset_dir='Basketball-Players-1',
    epochs=10,
    batch_size=4,
    grad_accum_steps=4,
    lr=1e-4,
    output_dir='Basket-Model'
)

Loading pretrain weights
TensorBoard logging initialized. To monitor logs, use 'tensorboard --logdir Basket-Model' and open http://localhost:6006/ in browser.
Not using distributed mode
git:
  sha: N/A, status: clean, branch: N/A

Namespace(num_classes=10, grad_accum_steps=4, amp=True, lr=0.0001, lr_encoder=0.00015, batch_size=4, weight_decay=0.0001, epochs=10, lr_drop=100, clip_max_norm=0.1, lr_vit_layer_decay=0.8, lr_component_decay=0.7, do_benchmark=False, dropout=0, drop_path=0.0, drop_mode='standard', drop_schedule='constant', cutoff_epoch=0, pretrained_encoder=None, pretrain_weights='rf-detr-base.pth', pretrain_exclude_keys=None, pretrain_keys_modify_to_load=None, pretrained_distiller=None, encoder='dinov2_windowed_small', vit_encoder_num_layers=12, window_block_indexes=None, position_embedding='sine', out_feature_indexes=[2, 5, 8, 11], freeze_encoder=False, layer_norm=True, rms_norm=False, backbone_lora=False, force_no_pretrain=False, dec_layers=3, dim_feedforward=2048, hidden_d

fatal: not a git repository (or any parent up to mount point /system)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


Get benchmark
Start training
Grad accum steps:  4
Total batch size:  16
LENGTH OF DATA LOADER: 21


Epoch: [0]  [ 0/21]  eta: 0:03:15  lr: 0.000100  class_error: 91.67  loss: 10.0164 (10.0164)  loss_ce: 1.5888 (1.5888)  loss_bbox: 0.3451 (0.3451)  loss_giou: 0.5859 (0.5859)  loss_ce_0: 1.5437 (1.5437)  loss_bbox_0: 0.3327 (0.3327)  loss_giou_0: 0.5991 (0.5991)  loss_ce_1: 1.5988 (1.5988)  loss_bbox_1: 0.3755 (0.3755)  loss_giou_1: 0.5738 (0.5738)  loss_ce_enc: 1.4878 (1.4878)  loss_bbox_enc: 0.3432 (0.3432)  loss_giou_enc: 0.6419 (0.6419)  loss_ce_unscaled: 1.5888 (1.5888)  class_error_unscaled: 91.6667 (91.6667)  loss_bbox_unscaled: 0.0690 (0.0690)  loss_giou_unscaled: 0.2929 (0.2929)  cardinality_error_unscaled: 3890.2500 (3890.2500)  loss_ce_0_unscaled: 1.5437 (1.5437)  loss_bbox_0_unscaled: 0.0665 (0.0665)  loss_giou_0_unscaled: 0.2996 (0.2996)  cardinality_error_0_unscaled: 3891.0000 (3891.0000)  loss_ce_1_unscaled: 1.5988 (1.5988)  loss_bbox_1_unscaled: 0.0751 (0.0751)  loss_giou_1_unscaled: 0.2869 (0.2869)  cardinality_error_1_unscaled: 3887.2500 (3887.2500)  loss_ce_enc_unsca

### 3. Quick Test of the Trained Model

The trained model is used to run a prediction on a sample image (`TEST.jpg`) that contains a basketball game scene.


In [2]:
model.predict('TEST.jpg')

Model is not optimized for inference. Latency may be higher than expected. You can optimize the model for inference by calling model.optimize_for_inference().


Detections(xyxy=array([[ 708.18164 ,  196.4009  ,  803.57416 ,  300.27127 ],
       [ 360.5881  ,  290.9437  ,  526.2546  ,  739.23065 ],
       [ 524.9138  ,  387.54086 ,  759.09424 ,  798.1892  ],
       [ 954.0385  ,  359.77963 , 1089.3196  ,  638.8316  ],
       [ 471.0634  ,  225.49442 ,  517.7854  ,  271.68228 ],
       [ 743.5514  ,  408.42294 ,  900.67346 ,  748.056   ],
       [ 818.6476  ,  554.15234 , 1012.37006 ,  799.9187  ],
       [  64.659065,  278.66968 ,  209.17209 ,  603.0442  ],
       [ 980.69806 ,  527.9076  , 1138.5411  ,  800.03894 ],
       [ 623.7248  ,  691.1232  ,  735.70667 ,  800.0969  ]],
      dtype=float32), mask=None, confidence=array([0.9279247 , 0.899186  , 0.8771306 , 0.86590713, 0.84771323,
       0.84472615, 0.81926185, 0.8147313 , 0.8042644 , 0.58313715],
      dtype=float32), class_id=array([2, 4, 4, 4, 1, 4, 4, 5, 4, 4]), tracker_id=None, data={}, metadata={})